In [1]:
!pip install Spotipy

In [2]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2
import random
from functools import reduce

In [3]:
!pip install -r requirements.txt

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util


cid ='48a1c43c43924406a14f4ec9d3583b66' # Client ID; copy this from your app created on beta.developer.spotify.com
secret = '45092c99dee94cab946442a479afe242' # Client Secret; copy this from your app
username = 'sanjsarda' # Your Spotify username
redirect_uri = 'https://example.com'

# Once the Authorisation is complete, we just need to `sp` to call the APIs
scope = 'user-top-read user-read-recently-played user-follow-read playlist-modify-public user-library-read playlist-read-collaborative'
token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [5]:
def get_tracks (sp,genre,popularity=50,limit=100):
    recos=sp.recommendations(seed_genres=genre,limit=limit, min_popularity=popularity)
    track_ids=[]
    for track in recos['tracks']:
        album_id=track['album']['id']
        album_tracks=sp.album_tracks(album_id)
        for track in album_tracks['items']:
            track_ids.append(track['id'])
    return track_ids

In [6]:
x=get_tracks(sp,['acoustic'])

In [7]:
len(x)

1133

In [8]:
results_recents= sp.current_user_recently_played(limit=25)

In [9]:
results_recents['items']

[{'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/69bG9tC62d8oTFC9aTTosn'},
      'href': 'https://api.spotify.com/v1/artists/69bG9tC62d8oTFC9aTTosn',
      'id': '69bG9tC62d8oTFC9aTTosn',
      'name': 'New West',
      'type': 'artist',
      'uri': 'spotify:artist:69bG9tC62d8oTFC9aTTosn'}],
    'available_markets': ['AD',
     'AE',
     'AG',
     'AL',
     'AM',
     'AO',
     'AR',
     'AT',
     'AU',
     'AZ',
     'BA',
     'BB',
     'BD',
     'BE',
     'BF',
     'BG',
     'BH',
     'BI',
     'BJ',
     'BN',
     'BO',
     'BR',
     'BS',
     'BT',
     'BW',
     'BY',
     'BZ',
     'CD',
     'CG',
     'CH',
     'CI',
     'CL',
     'CM',
     'CO',
     'CR',
     'CV',
     'CW',
     'CY',
     'CZ',
     'DE',
     'DJ',
     'DK',
     'DM',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FJ',
     'FM',
     'FR',
     'GA',
     'GB',
     

In [22]:
test=sp.current_user_top_tracks(limit=1, offset=0,time_range='medium_term')

In [38]:
sp.artist(test['items'][0]['artists'][0]['id'])['genres']

['alt z',
 'dark pop',
 'electropop',
 'indie electropop',
 'indie pop',
 'indie poptimism',
 'pop']

In [56]:
def get_user_profile(sp):
    results_favorite= sp.current_user_top_tracks(limit=25, offset=0,time_range='medium_term')
    results_recents= sp.current_user_recently_played(limit=25)
    
    track_name = []
    track_id = []
    artist = []
    artist_id = []
    album = []
    duration = []
    popularity = []
    genres= set()
    
    for i, items in enumerate(results_favorite['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        artist_id.append(items["artists"][0]["id"])
        genres.update(sp.artist(items["artists"][0]["id"])['genres'])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])
        
    for i, items in enumerate(results_recents['items']):
        track_name.append(items['track']['name'])
        track_id.append(items['track']['id'])
        artist.append(items['track']["artists"][0]["name"])
        artist_id.append(items['track']["artists"][0]["id"])
        genres.update(sp.artist(items['track']["artists"][0]["id"])['genres'])
        duration.append(items['track']["duration_ms"])
        album.append(items['track']["album"]["name"])
        popularity.append(items['track']["popularity"])
        
        
    df_user = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "artist_id": artist_id,
                             "duration": duration, 
                             "popularity": popularity})
    
    df_user.drop_duplicates(keep='first',inplace=True)
    return df_user, genres

In [57]:
user_profile, user_genres=get_user_profile(sp)

In [58]:
user_profile

,track_name,album,track_id,artist,artist_id,duration,popularity
0,Knew You,Here In Your Bedroom,0WfV9XwpkldKdjzw4qoyAd,Kailee Morgue,245PKF3hKjtxJKIG153kF0,185408,40
1,Flower Shops (feat. Morgan Wallen),Flower Shops (feat. Morgan Wallen),2ccuOtUjIyx3tPcsnpeBzJ,ERNEST,4GGfAshSkqoxpZdoaHm7ky,214405,79
2,Backseat (Kiss Me),Backseat (Kiss Me),7dbnw3cDgeDxOxJJ7QiAiD,Jutes,53fzjsJnjEKkA6TdncuIM4,166060,53
3,He Went To Jared (HARDY feat. Morgan Wallen),He Went To Jared (HARDY feat. Morgan Wallen),25uzAwyPekZtWzlLVvUYNS,HARDY,5QNm7E7RU2m64l6Gliu8Oy,213693,50
4,lie to me,lie to me,4f9IfFvBFm7Gsf1IVEudyf,Tate McRae,45dkTj5sMRSjrmBSBeiHym,177440,70
5,Taste of You,Taste of You,655sojH8GGEteZTBJOFyy7,Rezz,4aKdmOXdUKX07HVd3sGgzw,167606,64
6,One of Them Nights,One of Them Nights,4ks4jp4jGsmffhbUegLG6S,Chris Young,4BYxqVkZyFjtik7crYLg5Q,162520,38
7,Beggin',Conquest,4YeKAwFn69ehnciOfi7Sbf,Madcon,6c4sUNBgdonFJz8Kx2VsGz,218706,65
8,come over,After Midnight,6PGJZEvtlf2EJPu9VLqI3q,12AM,4FHtVj8v4il9h5HRQ4Ib3n,126087,15
9,The Reason,The Reason (15th Anniversary Deluxe),77loZpT5Y5PRP1S451P9Yz,Hoobastank,2MqhkhX4npxDZ62ObR5ELO,232800,82


In [61]:
len(user_genres)

51

In [62]:
def genre_consistency(sp,predicted_playlist_genres,user_genres):
    genre_consistency= len(predicted_playlist_genres.intersection(user_genres))/len(predicted_playlist_genres)
    return genre_consistency

In [14]:
def song_newness(sp, predicted_playlist):
    number_of_new_tracks=0
    for track in predicted_playlist:
        if track not in sp.current_user_saved_tracks() or track not in sp.current_user_recently_played(limit=50):
            number_of_new_tracks+=1
    return number_of_new_tracks*100/len(predicted_playlist)

In [60]:
def get_genres(sp, track_ids):
    genres= set()
    for track_id in track_ids:
        genres.update(sp.artist(track_id)['genres'])
    return genres
        